<a href="https://colab.research.google.com/github/evakaldy/Bootcamp_Final_Project/blob/main/00_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# # If you are importing notebooks, install the import-ipynb package:
!pip install import-ipynb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.0 MB/s eta 0:00:00


In [85]:
from google.colab import drive
import import_ipynb
import numpy as np

%run give_df_from_sql.ipynb # to make the function from colab work...
from give_df_from_sql import give_df # importing my colab notebook

%run data_transform_final_project.ipynb
from data_transform_final_project import data_transform, statistical_difference, plot_variables_pvalue

%run data_transform_for_random_forrest.ipynb
from data_transform_for_random_forrest import tranform_df_for_random_forest

%run random_forest.ipynb
from random_forest import single_random_forest

%run randomly_selected_client_succes_rate.ipynb
from randomly_selected_client_succes_rate import succes_rate_random

In [29]:
# Krok 1: Připojte Google Disk

drive.mount('/content/drive')
# my_project_path = '/content/drive/MyDrive/Colab Notebooks/INTENSIVE_part_offline/final_project/'
# Navigate to the project folder
%cd  '/content/drive/MyDrive/Colab Notebooks/INTENSIVE_part_offline/final_project/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/INTENSIVE_part_offline/final_project


In [67]:
## MERGE DATABASE & CONVERT to dataframe


# path to data on Google disk
db_path = '/content/drive/MyDrive/Colab Notebooks/INTENSIVE_part_offline/final_project/data.db'
merged_df = give_df(db_path)

Accounts
sqlite_sequence
Campaigns
Clients
Outcomes


In [68]:
## TRANSFROM DATA into cathegorical, logaritmic, binary...
merged_df2 = data_transform(merged_df)

   client_id  age           job  marital  education  balance in_default  \
0    4815378   58    management  married   tertiary   2143.0         no   
1    5217541   44    technician   single  secondary     29.0         no   
2    6654642   33  entrepreneur  married  secondary      2.0         no   
3    1421898   47   blue-collar  married    unknown   1506.0         no   
4    2387933   33       unknown   single    unknown      1.0         no   

  housing loan  duration  pdays  previous  campaign poutcome        y  
0     yes   no       261     -1         0         1  unknown  unknown  
1     yes   no       151     -1         0         1  unknown  unknown  
2     yes  yes        76     -1         0         1  unknown  unknown  
3     yes   no        92     -1         0         1  unknown  unknown  
4      no   no       198     -1         0         1  unknown  unknown  
JOB CATEGORY converted to number): 
{'high_income': 0, 'low_income': 1, 'middle_income': 2}

------------------------

In [ ]:
print(merged_df.head())

In [59]:
## REMOVE COLUMNS which are not numbers
only_nr_df = merged_df.drop(columns=['job', 'marital', 'job_category', 'education', 'housing', 'in_default', 'housing', 'loan', 'poutcome', 'y'], inplace=False)
# only_nr_df.head()

In [60]:
### DIVIDE into train_negative, train_positive and test
df_train_negative = only_nr_df[only_nr_df['y_nr'].isin([ 0])]
df_train_positive = only_nr_df[only_nr_df['y_nr'].isin([2])]
df_test = only_nr_df[only_nr_df['y_nr'].isin([1 ,3])]

# df_train_positive.head()

In [64]:
## CALCULATE THE STATICAL DIFFERENE between columns (for positive and negative)
p_value_df = statistical_difference(df_train_positive, df_train_negative)

column marital_nr is binary, chi2 18.477405856682516
column job_nr is binary, chi2 36.76973316255313
column education_nr is binary, chi2 15.1618699769161
column age_nr is binary, chi2 99.62996502477496
column pdays_nr is binary, chi2 153.4416542834824
column previous_nr is binary, chi2 117.388534999245
column campaign_nr is binary, chi2 9.271156489445623
column in_default_nr is binary, chi2 0.0
column housing_nr is binary, chi2 48.88462833144263
column loan_nr is binary, chi2 21.87182161746215
column poutcome_nr is binary, chi2 386.8773872252409
column education_job_nr is binary, chi2 1.0006728335460453
              Column        p-value  statistically_different
0                age   2.740020e-01                        0
1            balance   1.096141e-07                        1
2           duration  2.329297e-121                        1
3       duration_log  2.329297e-121                        1
4              pdays   5.717436e-24                        1
5           previous   

In [75]:
## PLOT the difference between positive and negative Y:
# each variable, before and after transformation

plot_variables_pvalue(df_train_positive, df_train_negative, p_value_df)

In [77]:
## PREPARE DATA FOR RANDOM FOREST ANALYSIS


df_all_for_randomForest = tranform_df_for_random_forest(only_nr_df)

In [78]:
### DIVIDE into train_negative, train_positive and test
df_train_for_randomForest = df_all_for_randomForest[df_all_for_randomForest['y_nr'].isin([ 0, 2])]
df_forPrediction_for_randomForest = df_all_for_randomForest[df_all_for_randomForest['y_nr'].isin([1 ,3])]

df_train_for_randomForest.columns

Index(['client_id', 'marital_nr', 'housing_nr', 'loan_nr', 'y_nr', 'job_nr_0',
       'job_nr_1', 'job_nr_2', 'age_nr_0', 'age_nr_1', 'age_nr_2',
       'pdays_nr_0', 'pdays_nr_1', 'pdays_nr_2', 'education_nr_0',
       'education_nr_1', 'education_nr_2', 'campaign_nr_0', 'campaign_nr_1',
       'previous_nr_0', 'previous_nr_1', 'poutcome_nr_0', 'poutcome_nr_1',
       'poutcome_nr_2', 'poutcome_nr_3'],
      dtype='object')

In [79]:
## RANDOM FOREST - single on
rf_model = single_random_forest(df_train_for_randomForest)

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1198
           2       0.54      0.18      0.27       159

    accuracy                           0.89      1357
   macro avg       0.72      0.58      0.60      1357
weighted avg       0.86      0.89      0.86      1357

Accuracy Score: 0.8857774502579219

-------------------------------------
Feature importance
marital_nr: 0.0992
housing_nr: 0.0984
loan_nr: 0.0552
job_nr_0: 0.0382
job_nr_1: 0.0344
job_nr_2: 0.0416
age_nr_0: 0.0328
age_nr_1: 0.0451
age_nr_2: 0.0610
pdays_nr_0: 0.0152
pdays_nr_1: 0.0388
pdays_nr_2: 0.0216
education_nr_0: 0.0369
education_nr_1: 0.0460
education_nr_2: 0.0455
campaign_nr_0: 0.0209
campaign_nr_1: 0.0196
previous_nr_0: 0.0149
previous_nr_1: 0.0121
poutcome_nr_0: 0.0434
poutcome_nr_1: 0.0309
poutcome_nr_2: 0.1364
poutcome_nr_3: 0.0119


In [83]:
####PREDICT

# Predict on the test set
y_pred = rf_model.predict(df_forPrediction_for_randomForest[features])


print()
# Count occurrences of 2
count = np.count_nonzero(y_pred == 2)
print(f"Occurrences of 2 (nr of most likely clients): {count}")
print()

# merge predicted value of y to the dataframe with test:
df_test['y_predict']  = y_pred

# print(" length of vector predicted" , y_pred.shape)
df_test.head()


Occurrences of 2 (nr of most likely clients): 1603



<ipython-input-83-ab2a36e3e865>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['y_predict']  = y_pred


,client_id,age,balance,duration,pdays,previous,campaign,job_nr,age_nr,pdays_nr,...,campaign_nr,marital_nr,previous_nr,in_default_nr,housing_nr,loan_nr,poutcome_nr,y_nr,education_job_nr,y_predict
0,4815378,58,2143.0,261,-1,0,1,0,1,0,...,0,1,0,0,1,0,3,1,2,0
1,5217541,44,29.0,151,-1,0,1,2,1,0,...,0,0,0,0,1,0,3,1,3,0
2,6654642,33,2.0,76,-1,0,1,0,1,0,...,0,1,0,0,1,1,3,1,1,0
3,1421898,47,1506.0,92,-1,0,1,2,1,0,...,0,1,0,0,1,0,3,1,2,0
4,2387933,33,1.0,198,-1,0,1,1,1,0,...,0,0,0,0,0,0,3,1,1,0


In [ ]:
## SELECT the 1000 MOST BENEFICIAL clients that are likely to have y=yes
## Save the client IDs to a file named second_phase_target.csv.
## This file must contain exactly one column named "target".

# Filter rows where y_predict == 2
client_to_say_yes = df_test[df_test['y_predict'] == 2]

# Select the top 1000 rows with the highest balance
top_1000_client_to_say_yes = client_to_say_yes.nlargest(1000, 'balance')

# Keep only the "balance" column and rename it to "target"
output_df = top_1000_client_to_say_yes[['client_id']].rename(columns={'client_id': 'target'})

# Save to CSV
output_df.to_csv('second_phase_target.csv', index=False)

print("------------------------------------")
print("HERE IS SAVED THE RESULT")
print("--------'second_phase_target.csv'----------")
# Print the result
print(output_df)